## Data Processing Stage-2 [Preparing Tefla ready data]

This Notebook runs after the DataProcessing-1.ipynb notebook. This notebook creates the csv file with consists of nii files name and its corresponding label. Then it prepares the data in the tefla ready format, which the adequate distribution of data between training, validation, and test for all the categories.

In [ ]:
import os
import pandas as pd
import math
import numpy as np
from PIL import Image
import glob
import sys

In [ ]:
classes = ["Alzheimer", "MCI", "Normal"]
#sourceDir = '/home/nagdev/work/siddhant/data/data/'
#sourceDir = '/home/h_hack/work/pywork/ML/datasets/Alzheimer_detect/fsl_preprocessed/images/'
#path_tocsvfile = '/home/h_hack/work/pywork/ML/datasets/Alzheimer_detect/fsl_preprocessed/'
#path_tocsvfile = '/home/nagdev/work/siddhant/data/raw/'
#sourceDir = '/home/ec2-user/final_data/images/'
#path_tocsvfile = '/home/ec2-user/final_data/'
sourceDir = '/home/ubuntu/Select_data/images/'
path_tocsvfile = '/home/ubuntu/Select_data/'

# 0 represents Alzheimer class, 1 represents MCI class, 2 represents Normal class.
count = 0
with open(path_tocsvfile + 'all.csv','a') as o:
    o.write('nii' + ',' + 'label' +'\n')
    for class_ in classes:
        images_path = os.path.join(os.path.join(sourceDir,class_), '*.nii')
        print 'Working on: ', images_path
        for file_ in glob.glob(images_path):
            o.write(file_ + ',' + str(count) +'\n')
        count += 1
        print count

In [ ]:
import os
import pandas as pd
import math
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
# script to create a tefla compatible data dir for training and validation data
#sourceDir = '/home/nagdev/work/siddhant/data/raw/'
#destDir = '/home/nagdev/work/siddhant/data/raw/processed/'
#sourceDir = '/home/ec2-user/final_data/'
#destDir = '/home/ec2-user/final_data/processed/'
sourceDir = '/home/ubuntu/Select_data/'
destDir = '/home/ubuntu/Select_data/processed/'

categories_picked_for_validation_data = [0,1]
#categories_picked_for_validation_data = [0,1,2] # if we use all three categories alzheimer, mci and normal
validation_data_percentage = 10

def create_tefla_data(source_dir,destination_dir,validation_categories,validation_percentage):
    #if os.path.exists(destination_dir):
    #    shutil.rmtree(destination_dir)

    training_dir = destination_dir + 'training_64/'
    validation_dir = destination_dir + 'validation_64/'
    
    try:
        os.makedirs(training_dir)
    except:
        pass
    try:
        os.makedirs(validation_dir)
    except:
        pass
    
    labels = pd.read_csv( source_dir + 'all.csv')
    validation_dict = {}

    for c in validation_categories:
        validation_dict[c] = []
    # now test_set has some nii files inside it from each class

    #validation set creation
    validation_set = []

    for i, rows in labels.iterrows():
        if validation_dict.has_key(rows['label']):
            validation_dict[rows['label']].append(rows['nii'])

    for l in validation_dict:
        np.random.seed(0)
        n = math.ceil(validation_percentage * len(validation_dict[l]) / 100.0)
        random_array = np.random.choice(validation_dict[l],int(n))
        validation_set = validation_set + random_array.tolist()
    # add some of the nii files to validation set which are not part of test set.

    training_images = []
    training_labels = []
    validation_images = []
    validation_labels =[]
    count = 0
    for i, rows in labels.iterrows():
        if rows['nii'] in validation_set:
            #print rows['nii']
            for file_ in glob.glob(os.path.join(rows['nii'], '*.jpg')):
                validation_images.append(os.path.basename(file_).replace('.jpg', ''))
                validation_labels.append(rows['label'])
                count = count + 1
                process_and_save_image(file_, validation_dir + os.path.basename(file_).replace('.jpg','') + ".jpg")
        else:
            #print rows['nii']
            for file_ in glob.glob(os.path.join(rows['nii'], '*.jpg')):
                count = count + 1
                training_images.append(os.path.basename(file_).replace('.jpg', ''))
                training_labels.append(rows['label'])
                process_and_save_image(file_, training_dir + os.path.basename(file_).replace('.jpg','') + ".jpg")
        sys.stdout.write("\r" + str(count))
                
    header = ['image', 'label']

    # saving training csv
    training_out = np.column_stack((training_images, training_labels))
    training_out = np.row_stack((header, training_out))
    np.savetxt(destination_dir + 'training_labels.csv', training_out, delimiter=',', fmt='%s')

    # saving validation csv
    validation_out = np.column_stack((validation_images, validation_labels))
    validation_out = np.row_stack((header, validation_out))
    np.savetxt(destination_dir + 'validation_labels.csv', validation_out, delimiter=',', fmt='%s')

def process_and_save_image(source_path,destination_path):
    img = resize(source_path, 64)
    img.save(destination_path, quality=100)

def resize(fname, target_size):
    # print('Processing image: %s' % fname)
    img = Image.open(fname)
    return img

#calling method
create_tefla_data(sourceDir,destDir,categories_picked_for_validation_data,validation_data_percentage)

